# Initial Setup

In [1]:
# basic imports 

import os # OS e.g directory structure
import sys
import numpy as np # linear algebra
import scipy as sc  # scientific computing
import pandas as pd # data processing, file I/O
import seaborn as sns  # visualization
import matplotlib.pyplot as plt # visualization
import warnings
warnings.filterwarnings("ignore")

In [27]:
# Spark related imports

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


# Data Exploration

In [3]:
! echo "Oct-2019"
! head -n 5 dataset/2019-Oct.csv
! tail -n 5 dataset/2019-Oct.csv
! echo "Nov-2019"
! head -n 5 dataset/2019-Nov.csv
! tail -n 5 dataset/2019-Nov.csv

Oct-2019
event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("abd_classification").getOrCreate()

In [22]:
sales_oct = spark.read.csv("dataset/2019-Oct-small.csv", header="true", inferSchema="true", sep=",")
sales_nov = spark.read.csv("dataset/2019-Nov-small.csv", header="true", inferSchema="true", sep=",")

In [19]:
sales_oct_small, _ = sales_oct.randomSplit([0.005, 0.995], 42)
sales_nov_small, _ = sales_nov.randomSplit([0.005, 0.995], 42)

In [24]:
sales_oct.printSchema()
so = sales_oct.count()

sales_nov.printSchema()
sv = sales_nov.count()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [25]:
print(so)
print(sv)

9999
9999


In [32]:
sales_oct.select("user_id").show(truncate=False)

+---------+
|user_id  |
+---------+
|541312140|
|554748717|
|519107250|
|550050854|
|535871217|
|512742880|
|555447699|
|550978835|
|520571932|
|537918940|
|555158050|
|530282093|
|555444559|
|541312140|
|555446365|
|513642368|
|537192226|
|519528062|
|550050854|
|535871217|
+---------+
only showing top 20 rows



In [33]:
sales_nov.show(20, truncate=False)

+-----------------------+----------+----------+-------------------+--------------------------------+--------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                   |brand   |price |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+--------------------------------+--------+------+---------+------------------------------------+
|2019-11-01 00:00:00 UTC|view      |1003461   |2053013555631882655|electronics.smartphone          |xiaomi  |489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|
|2019-11-01 00:00:00 UTC|view      |5000088   |2053013566100866035|appliances.sewing_machine       |janome  |293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|
|2019-11-01 00:00:01 UTC|view      |17302664  |2053013553853497655|null                            |creed   |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|
|2019-11-01 00:0

In [28]:
sales_oct.describe()

DataFrame[summary: string, event_time: string, event_type: string, product_id: string, category_id: string, category_code: string, brand: string, price: string, user_id: string, user_session: string]

In [43]:
# df_users.select(col("Age")).where(col("Age") == "NULL").count()
# https://stackoverflow.com/questions/40163106/cannot-find-col-function-in-pyspark
#sales_oct.select("brand").where(F.col("brand") == "null").count()
#for column in sales_oct.columns:
#    value = sales_oct.select(F.col(column)).where(F.col(column) == "NULL").count()
#    print("Null number "+str(column)+ "=" +str(value))
sales_nov.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sales_nov.columns]
   ).show()
sales_oct.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sales_oct.columns]
   ).show()

3741
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|         0|         0|         0|          0|         3741| 1805|    0|      0|           0|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|         0|         0|         0|          0|         3277| 1442|    0|      0|           0|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



In [38]:
sales_oct.describe("price").show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|             9999|
|   mean|294.0986068606829|
| stddev|365.5235340279953|
|    min|              0.0|
|    max|          2574.07|
+-------+-----------------+



In [41]:
sales_oct.select('product_id', 'category_id', 'brand', 'price').where(col('price')==0).show()

+-------+--------------------+--------------------+-----+-----+
|summary|          product_id|         category_id|brand|price|
+-------+--------------------+--------------------+-----+-----+
|  count|                  11|                  11|    0|   11|
|   mean|2.4783409454545453E7|2.070040346477199...| null|  0.0|
| stddev|1.7841158231601987E7|3.788220907754455...| null|  0.0|
|    min|             1003507| 2053013553526341921| null|  0.0|
|    max|            53000001| 2146660886926852416| null|  0.0|
+-------+--------------------+--------------------+-----+-----+



Groupby user session using market basket since we have not user valuation

In [50]:
# https://stackoverflow.com/questions/48406304/groupby-and-concat-array-columns-pyspark
sales_oct.groupBy('user_session').agg(collect_list('product_id').alias('products')).show(20, truncate=False)

+------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_session                        |products                                                                                                                                              |
+------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|051fa0c1-a1c1-4276-8d57-fc0f3f5e2933|[3600661, 3600666, 3601537]                                                                                                                           |
|2af9b570-0942-4dcd-8f25-4d84fba82553|[4804055, 4804055, 4804055]                                                                                                                           |
|75f3f39e-cc56-4564-8b1b-5db49ca95314|[12707153, 1

In [49]:
sales_oct.groupBy('user_session').agg(collect_list('brand').alias('brand')).show(30, truncate=False)

+------------------------------------+--------------------------------------------------------+
|user_session                        |brand                                                   |
+------------------------------------+--------------------------------------------------------+
|051fa0c1-a1c1-4276-8d57-fc0f3f5e2933|[samsung, samsung, samsung]                             |
|2af9b570-0942-4dcd-8f25-4d84fba82553|[apple, apple, apple]                                   |
|75f3f39e-cc56-4564-8b1b-5db49ca95314|[bars]                                                  |
|78c0b329-af93-44fa-966d-08d7c69a37b1|[brw]                                                   |
|85881243-7b33-4098-8418-a2fc9a3c2e05|[wingoffly, belecoo, wingoffly]                         |
|898ff23f-fc70-4239-879d-4ef9226cd6db|[yokohama]                                              |
|90551c19-3a04-45bb-9f8b-2f06f793a95c|[samsung]                                               |
|a6b41834-75dd-4cd5-ace6-cbe1a4fe54df|[l